# English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

## English translation pending

See the `japanese` branch for the original content.


In [ ]:
import re
import numpy as np
import pandas as pd
import plotly.express as px

from src.base_flare_detector import BaseFlareDetector
from src.flarepy_EK_Dra import FlareDetector_EK_Dra
from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DIR_EK_DRA = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
DATA_DIR_DS_TUC_A = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_DIR_V889_HER = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DATA_DIR_EK_DRA:", DATA_DIR_EK_DRA)
print("DATA_DIR_DS_TUC_A:", DATA_DIR_DS_TUC_A)
print("DATA_DIR_V889_HER:", DATA_DIR_V889_HER)


In [ ]:
def extract_sector(data_name: str) -> int:
    """English translation pending TESS English translation pending (sXXXX) English translation pending。"""
    m = re.search(r"-s(\d{4})-", data_name)
    return int(m.group(1)) if m else -1


def run_detectors_for_all_stars(max_files_per_star: int = 3) -> pd.DataFrame:
    """EK Dra / DS Tuc A / V889 Her English translation pending、English translation pending FITS English translation pending。"""
    rows: list[dict] = []

    star_configs = [
        ("EK Dra", DATA_DIR_EK_DRA, FlareDetector_EK_Dra),
        ("DS Tuc A", DATA_DIR_DS_TUC_A, FlareDetector_DS_Tuc_A),
        ("V889 Her", DATA_DIR_V889_HER, FlareDetector_V889_Her),
    ]

    for star_name, data_dir, Detector in star_configs:
        if not data_dir.exists():
            print(f"[warning] {star_name} English translation pending {data_dir} English translation pending。")
            continue

        fits_files = sorted(data_dir.glob("*.fits"))[:max_files_per_star]
        if not fits_files:
            print(f"[warning] {star_name} English translation pending FITS English translation pending。")
            continue

        for fpath in fits_files:
            print(f"Processing {star_name}: {fpath.name}")
            det = Detector(file=str(fpath), process_data=True)

            if det.precise_obs_time <= 0:
                flare_ratio = np.nan
                energy_ratio = np.nan
            else:
                flare_ratio = det.flare_number / det.precise_obs_time
                energy_ratio = det.sum_flare_energy / det.precise_obs_time

            rows.append(
                {
                    "star": star_name,
                    "data_name": det.data_name,
                    "sector": extract_sector(det.data_name or ""),
                    "flare_ratio": flare_ratio,
                    "energy_ratio": energy_ratio,
                    "starspot": det.starspot,
                    "starspot_ratio": det.starspot_ratio,
                    "rotation_period": det.per,
                    "rotation_period_err": det.per_err,
                }
            )

    df = pd.DataFrame(rows)
    return df


summary_df = run_detectors_for_all_stars(max_files_per_star=2)
summary_df.head()


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
ek_df = summary_df[summary_df["star"] == "EK Dra"].copy()

if ek_df.empty:
    print("EK Dra English translation pending。English translation pending summary_df English translation pending。")
else:
    ek_df = ek_df.sort_values("sector")

    fig = px.scatter(
        ek_df,
        x="sector",
        y="flare_ratio",
        size="energy_ratio",
        hover_data=["data_name"],
        labels={"sector": "TESS Sector", "flare_ratio": "Flare Rate [1/day]"},
        title="EK Dra: English translation pending (English translation pending)",
    )
    fig.update_yaxes(type="log")
    fig.show()


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
if summary_df.empty:
    print("summary_df English translation pending。English translation pending run_detectors_for_all_stars English translation pending。")
else:
    fig = px.scatter(
        summary_df,
        x="rotation_period",
        y="flare_ratio",
        color="star",
        size="starspot_ratio",
        hover_data=["sector", "data_name"],
        labels={
            "rotation_period": "Rotation Period [day]",
            "flare_ratio": "Flare Rate [1/day]",
            "starspot_ratio": "Starspot Filling Factor (relative)",
        },
        title="English translation pending",
    )
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.show()


## English translation pending

See the `japanese` branch for the original content.
